In [2]:
import os
import re
import sys
import time
import warnings
from datetime import datetime

import pickle
import pymysql
import requests
from tqdm.auto import tqdm

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

cur_dir = os.path.dirname(os.path.realpath("__file__"))
root = os.path.abspath(os.path.join(cur_dir, os.pardir))
src = os.path.join(root, 'src')
sys.path.append(src)
tbl_cache = os.path.join(root, 'tbl_cache')

from database.conn import AccessDatabase
from crawling.crawler import get_url, json_iterator, get_headers
today = datetime.today().strftime('%y%m%d')

db_glamai = AccessDatabase('glamai')
db_jangho = AccessDatabase('jangho')

warnings.filterwarnings("ignore")

###
### Sephora Review Check
---

In [ ]:
# Check crawling result
txt_data_path = os.path.join(tbl_cache, "txt_data.txt")
with open(txt_data_path, "rb") as f:
    txt_data = pickle.load(f)
    
error_path = os.path.join(tbl_cache, "error.txt")
with open(error_path, "rb") as f:
    error = pickle.load(f)


columns = ['product_code', 'product_id', 'rating', 'skin_type', 'eye_color', 'skin_concerns', 'hair_color', 'skin_tone', 'age', 'title', 'txt_data', 'positive_count', 'write_time', 'regist_date']
rev_df = pd.DataFrame(txt_data, columns=columns)

error_df = pd.DataFrame(error, columns=['product_code', 'product_url', 'note'])
error_df_cnt = error_df.groupby('note').count()

rev_df.groupby('product_code').count()

print(\
    f"\n\nproduct counts: {len(rev_df.product_code.unique())}\n\
    product review counts: {len(rev_df)}\n\
    reviews that already exist: {error_df_cnt.iloc[0, 0]}\n\
    review does not exist: {error_df_cnt.iloc[1, 0]}"
)

###
### Sephora Status Check
---

In [ ]:
# Check status

verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
df_list = []
for vertical in verticals:
    query = f'select is_use, count(*) as count, "{vertical}" as vertical from `sephora_{vertical}_data_status` group by is_use;'
    conn, curs = db_glamai._connect()
    curs.execute(query)
    data = curs.fetchall()
    df = pd.DataFrame(data)
    print(df, '\n\n')
    df_list.append(df)

curs.close()
conn.close()    
status_df = pd.concat(df_list)

In [ ]:
bak_date = input("Enter the date (가장 최근에 세포라 status 업데이트를 시행한 날짜)[ ex) `221203` ] :  ")

In [ ]:
# Check product counts
verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
conn, curs = db_glamai._connect()
datas = []
current, paste = 0, 0
for vertical in verticals:
    query = f'''\
    select 'sephora_{vertical}_data_status' as tbl, count(*) as cnt from sephora_{vertical}_data_status union\
    select 'sephora_{vertical}_data_status_bak_{bak_date}', count(*) as cnt from sephora_{vertical}_data_status_bak_{bak_date};'''
    curs.execute(query)
    data = curs.fetchall()
    df = pd.DataFrame(data)
    print(df, '\n\n')
    datas += data
    current += df.iloc[0, 1]
    paste += df.iloc[1, 1]
    
curs.close()
conn.close()    

df = pd.DataFrame(datas)
print(f' - Current: {current}\n - Paste: {paste}\n - Change: {current - paste}')


In [ ]:
# Check product counts where is_use=1
verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
conn, curs = db_glamai._connect()
datas = []
current, paste = 0, 0
for vertical in verticals:
    query = f'''\
    select 'sephora_{vertical}_data_status' as tbl, count(*) as cnt from sephora_{vertical}_data_status where is_use=1 union\
    select 'sephora_{vertical}_data_status_bak_{bak_date}', count(*) as cnt from sephora_{vertical}_data_status_bak_{bak_date} where is_use=1;'''
    curs.execute(query)
    data = curs.fetchall()
    df = pd.DataFrame(data)
    print(df, '\n\n')
    datas += data
    current += df.iloc[0, 1]
    paste += df.iloc[1, 1]
    
curs.close()
conn.close()    

df = pd.DataFrame(datas)
print(f' - Current: {current}\n - Paste: {paste}\n - Change: {current - paste}')

###
### Amazon Status Check & Update
---

In [3]:
from affiliate.amazon import get_data_amazon
_date = input("Enter the date [ ex) `221203` ] :  ")

tbl = f'affiliate_price_update_amazon_{_date}'
upload_df = db_jangho.get_tbl(tbl)

upload_df.groupby('is_use').count()



`affiliate_price_update_amazon_221216` Import Time: 0.2sec


,product_code,item_no,affiliate_type,affiliate_url,affiliate_image,price,sale_price,is_sale,regist_date,update_date
is_use,,,,,,,,,,
0,2050,2050,2050,2050,2049,2050,2050,2050,2050,2050
1,2641,2641,2641,2641,2641,2641,2641,2641,2641,2641


In [4]:
upload_df[upload_df.is_use==1].groupby('is_sale').count()

,product_code,item_no,affiliate_type,affiliate_url,affiliate_image,price,sale_price,is_use,regist_date,update_date
is_sale,,,,,,,,,,
0,1862,1862,1862,1862,1862,1862,1862,1862,1862,1862
1,779,779,779,779,779,779,779,779,779,779


In [5]:
# Check & Re-Crawling
for idx in upload_df[(upload_df.is_use==-1) | (upload_df.is_use==2)].index:
    
    product_code = upload_df.loc[idx, "product_code"]
    item_no = upload_df.loc[idx, "item_no"]
    affiliate_type = upload_df.loc[idx, "affiliate_type"]
    url = upload_df.loc[idx, "affiliate_url"]
    crawled = get_data_amazon(url, window=True, image=True)
    price, sale_price, is_sale, is_use = crawled[3:7]

    if is_use == 1:
        query = f"""\
        UPDATE {tbl} \
        SET price={price}, sale_price={sale_price}, is_sale={is_sale}, is_use={is_use} \
        WHERE product_code='{product_code}' and item_no={item_no} and affiliate_type='{affiliate_type}';\
        """
        db_jangho._execute(query)

In [ ]:
query = f'''
update glamai.affiliate_price as a
join jangho.affiliate_price_update_amazon_{_date} as b 
on a.product_code = b.product_code and a.item_no = b.item_no and a.affiliate_type = b.affiliate_type
set a.price = b.price, a.sale_price = b.sale_price, a.is_sale = b.is_sale, a.is_use = b.is_use, a.regist_date = b.regist_date, a.update_date = b.update_date
where b.is_use!=-1 and b.is_use!=2;'''

conn, curs = db_jangho._connect()
curs.execute(query)
conn.commit()
curs.close()
conn.close()

# Check query
query = 'select * from affiliate_price where is_use=1 and is_sale=0 and sale_price=0;'
conn, curs = db_glamai._connect()
curs.execute(query)
data = curs.fetchall()

if len(data) == 0:
    print("\n\nComplete amazon status data!")
else:
    print("\n\nError: Check status!")
    
curs.close()
conn.close()

###
### Ulta Status Check & Update
---

In [ ]:
_date = input("Enter the date [ ex) `221203` ] :  ")

tbl = f'affiliate_price_update_ulta_{_date}'
upload_df = db_jangho.get_tbl(tbl)
upload_df.groupby('is_use').count()

In [ ]:
upload_df[upload_df.is_use==1].groupby('is_sale').count()

In [ ]:
query = f'''
update glamai.affiliate_price as a
join jangho.affiliate_price_update_ulta_{_date} as b 
on a.product_code = b.product_code and a.item_no = b.item_no and a.affiliate_type = b.affiliate_type
set a.price = b.price, a.sale_price = b.sale_price, a.is_sale = b.is_sale, a.is_use = b.is_use, a.regist_date = b.regist_date, a.update_date = b.update_date
where b.is_use!=-1;'''

conn, curs = db_jangho._connect()
curs.execute(query)
conn.commit()
curs.close()
conn.close()
# Check query

query = 'select * from affiliate_price where is_use=1 and is_sale=0 and sale_price=0;'
conn, curs = db_glamai._connect()
curs.execute(query)
data = curs.fetchall()
curs.close()
conn.close()

if len(data) == 0:
    print("Complete update ulta sale!")